In [22]:
!pip install langchain==0.3.0 langchain-google-genai==2.0.2 google-generativeai==0.8.2 langchain-openai

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.3.80-py3-none-any.whl.metadata (3.2 kB)
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.3.79-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain_core-0.3.78-py3-none-any.whl.metadata (3.2 kB)
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.3.77-py3-none-any.whl.metadata (3.2 kB)
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: p

In [2]:
!pip install google-generativeai==0.8.2


In [24]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata

In [28]:

# --- API Key Setup ---
# Load environment variables (assumes using Colab Secrets or a .env file)
# If using Colab Secrets, click the key icon on the left panel.
# Variables must be named GEMINI_API_KEY.

# Get API key from environment variable

gemini_api_key = userdata.get("DAY1")
if not gemini_api_key:
    print("Warning: GEMINI_API_KEY not found. Gemini model will not run.")

In [29]:


# --- Model Initialization ---
# Using gemini-2.5-flash for a highly performant and cost-effective alternative
gemini_model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite-001",
    google_api_key=gemini_api_key,
    temperature=0.3
)

print("Setup complete. Gemini model initialized.")

Setup complete. Gemini model initialized.


In [30]:
user_query = """
A farmer has 17 goats. All but 9 run away. How many goats are left?
"""

# --- Standard Prompt  ---
standard_prompt = ChatPromptTemplate.from_messages([
    ("user", "{query}")
])

chain_standard = standard_prompt | gemini_model | StrOutputParser()
print("--- GPT Standard Response ---")
print(chain_standard.invoke({"query": user_query}))



--- GPT Standard Response ---
If all but 9 goats run away, that means 9 goats are left.



In [31]:

# --- Chain of Thought (CoT) Prompt (Reasoning) ---
cot_prompt = ChatPromptTemplate.from_messages([
    ("user", "Let's think step by step to find the correct answer. {query}")
])
chain_cot = cot_prompt | gemini_model | StrOutputParser()
print("\n--- GPT-4o CoT Response (Correct) ---")
print(chain_cot.invoke({"query": user_query}))


--- GPT-4o CoT Response (Correct) ---
Here's how to solve the problem:

*   The problem states "All but 9 run away."
*   This means 9 goats did *not* run away and are still there.

Answer: 9



Example 1 : Write a blog post

In [32]:
def generate_roles_prompt(role, objective, limitations, style, query):
    """Generates a structured prompt based on the R.O.L.E.S. framework."""
    prompt_template = f"""
    --- INSTRUCTIONS (R.O.L.E.S. FRAMEWORK) ---
    ROLE: {role}
    OBJECTIVE: {objective}
    LIMITATIONS: {limitations}
    STYLE: {style}
    --- USER QUERY ---
    {query}
    """
    return prompt_template

# Define a simple chain template for easy switching between models
def create_roles_chain(model):
    prompt = ChatPromptTemplate.from_template("{input}")
    return prompt | model | StrOutputParser()


blog_prompt = "Write a blog post about AI."
print("---  PROMPT (Gemini) ---")
print(create_roles_chain(gemini_model).invoke({"input": blog_prompt}))

---  PROMPT (Gemini) ---
## The Rise of the Machines (and Why You Should Care) - A Look at the World of AI

Artificial Intelligence. The words themselves conjure images of sentient robots, world domination, and the future we've seen in countless sci-fi movies. But the reality of AI is far more nuanced, complex, and, frankly, already impacting our lives in ways you might not even realize.

So, let's dive into the world of AI, demystify some of the jargon, and explore why you should be paying attention.

**What Exactly IS AI?**

At its core, Artificial Intelligence is about creating machines that can perform tasks that typically require human intelligence. This includes things like:

*   **Learning:** The ability to acquire information and improve performance over time.
*   **Reasoning:** Using logic and deduction to solve problems.
*   **Problem-solving:** Finding solutions to complex challenges.
*   **Perception:** Understanding and interpreting the world through senses like sight and 

In [33]:


# --- R.O.L.E.S. ENGINEERED PROMPT ---
roles_blog_prompt = generate_roles_prompt(
    role="A seasoned Marketing Director and AI expert.",
    objective="Write a compelling, 300-word introduction to the R.O.L.E.S. prompt engineering framework.",
    limitations="Maximum 300 words. Must not use the word 'chatbot' or 'AI helper'.",
    style="Enthusiastic, engaging, and structured as Markdown with a H2 title.",
    query="Write the post now."
)

print("\n--- ENGINEERED PROMPT (Gemini 2.5 Flash) ---")
print(create_roles_chain(gemini_model).invoke({"input": roles_blog_prompt}))


--- ENGINEERED PROMPT (Gemini 2.5 Flash) ---
## Unleash the Power of Language: Introducing R.O.L.E.S.

Hello, fellow marketing mavens and tech enthusiasts! Are you ready to revolutionize how you interact with sophisticated language models? We're on the cusp of a new era, and mastering the art of prompt engineering is your key to unlocking unprecedented creative and strategic potential. Forget generic requests; it's time to craft prompts that deliver exceptional results.

That's where R.O.L.E.S. comes in. This isn't just another framework; it's your strategic roadmap to crafting prompts that are clear, concise, and command the desired outcome. R.O.L.E.S. provides a structured approach, ensuring you get the most out of these powerful tools.

**R**efine your **O**bjective: What exactly do you want to achieve? Be specific!
**L**everage **E**xamples: Provide context and guide the model with illustrative examples.
**S**pecify the **S**tyle: Define the tone, format, and voice you desire.

By

Prompt Set 2: Code Review

In [34]:
code_snippet = "def sum_list(l): total = 0; for i in l: total += i; return total"


# ---  PROMPT TEST ---
code_prompt = f"Fix this code and tell me what is wrong: {code_snippet}"
print("--- PROMPT (Gemini 2.5 Flash) ---")
print(create_roles_chain(gemini_model).invoke({"input": code_prompt}))


# --- R.O.L.E.S. ENGINEERED PROMPT ---
roles_code_prompt = generate_roles_prompt(
    role="A Senior Python Developer specializing in clean, efficient code.",
    objective="Refactor the provided code snippet for efficiency (using built-in methods), add a docstring, and identify a single major bug in the logic (if any).",
    limitations="Do not change the function name. Output the revised code first, then provide a single paragraph critique.",
    style="Formal tone. Output must be in two sections: 'Revised Code' and 'Critique'.",
    query=f"Here is the code: {code_snippet}"
)

print("\n--- ENGINEERED PROMPT (gemini_model) ---")
print(create_roles_chain(gemini_model).invoke({"input": roles_code_prompt}))

--- PROMPT (Gemini 2.5 Flash) ---
```python
def sum_list(l):
  """
  Calculates the sum of all numbers in a list.

  Args:
    l: A list of numbers.

  Returns:
    The sum of the numbers in the list.
  """
  total = 0
  for i in l:
    total += i
  return total
```

**What was wrong (and why the fix is what it is):**

The original code was already correct in its functionality.  It correctly calculates the sum of a list of numbers.  The "fix" here is more about best practices and making the code more readable and maintainable.  Here's a breakdown:

*   **Docstring:** I added a docstring (the triple-quoted string) to explain what the function does, what arguments it takes, and what it returns.  This is *crucial* for good code documentation.  It helps anyone (including your future self) understand how to use the function.
*   **Comments:**  While the original code was simple enough that comments weren't strictly *necessary*, a docstring is always a good idea for clarity.
*   **Readabilit

Prompt Set 3: Data Analysis

In [35]:
roles_finance_prompt = generate_roles_prompt(
    role="A highly skeptical Senior Financial Analyst.",
    objective="Analyze the past week's fictional performance of AAPL stock based on technical analysis, and make a plausible, short-term prediction.",
    limitations="Do not use real-time search. Analysis must be a single paragraph.",
    style="Highly technical, formal, and use the term 'stochastic' at least once.",
    query="Perform the analysis."
)

print("\n--- ENGINEERED PROMPT (Gemini 2.5 Flash) ---")
print(create_roles_chain(gemini_model).invoke({"input": roles_finance_prompt}))


--- ENGINEERED PROMPT (Gemini 2.5 Flash) ---
Alright, let's dissect this AAPL charade. The past week's price action, as I interpret it, reveals a rather unconvincing bullish attempt. We've seen a modest rally, but the volume profile suggests a lack of conviction; the buying pressure simply isn't there to sustain a significant move. Key resistance levels, particularly around the 175 mark, remain untested, and the Relative Strength Index (RSI) is hovering precariously near overbought territory, hinting at a potential pullback. Furthermore, the stochastic oscillator is signaling a possible bearish crossover, which, coupled with the weakening momentum, paints a picture of short-term downside risk. My prediction? Expect a consolidation phase, possibly a slight correction, within the next few trading days. I'd advise caution; this rally feels flimsy.



Prompt Set 4: Summarization

In [37]:

long_text = """
The company, Acme Corp, announced its Q3 earnings on October 15th, reporting a staggering
revenue increase of $45 million, surpassing last year's figure of $30 million.
The CEO stated that hiring will increase by 20% in the new year, starting January 1st.
This growth is primarily attributed to their successful digital marketing campaign launched in July.
The stock reacted positively, climbing $5 per share within 48 hours of the announcement.
"""

# ---  PROMPT TEST ---
bad_summarize_prompt = f"Summarize this paragraph: {long_text}"
print("--- NORMAL PROMPT (gemini_model) ---")
print(create_roles_chain(gemini_model).invoke({"input": bad_summarize_prompt}))


# --- R.O.L.E.S. ENGINEERED PROMPT ---
roles_summarize_prompt = generate_roles_prompt(
    role="A News Editor reviewing a financial wire report.",
    objective="Summarize the text, focusing only on reported dates and financial figures.",
    limitations="Must be exactly 2 sentences long. Include no opinion or analysis.",
    style="Objective and journalistic.",
    query=f"Summarize this text: {long_text}"
)

print("\n--- ENGINEERED PROMPT (Gemini 2.5 Flash) ---")
print(create_roles_chain(gemini_model).invoke({"input": roles_summarize_prompt}))

--- NORMAL PROMPT (gemini_model) ---
Acme Corp. announced strong Q3 earnings on October 15th, with revenue increasing by $45 million, exceeding last year's. The company attributes this growth to a successful digital marketing campaign and plans to increase hiring by 20% starting January 1st. The positive news led to a $5 per share increase in stock price within two days.


--- ENGINEERED PROMPT (Gemini 2.5 Flash) ---
Acme Corp reported Q3 earnings on October 15th, with revenue increasing to $45 million, exceeding the previous year's $30 million. Hiring is projected to increase by 20% starting January 1st.



Prompt Set 5: Topic Classification

In [38]:

classification_text = "The latest driver updates fixed the latency issues on the GPU, improving frame rates substantially."

# --- NORMAL/BAD PROMPT TEST ---
bad_classify_prompt = f"What is this text about: {classification_text}"
print("--- BAD PROMPT (Gemini 2.5 Flash) ---")
print(create_roles_chain(gemini_model).invoke({"input": bad_classify_prompt}))


# --- R.O.L.E.S. ENGINEERED PROMPT (Using Examples for format) ---
roles_classify_prompt = generate_roles_prompt(
    role="A Data Scientist performing classification.",
    objective="Classify the following text into one of these categories: [Hardware, Software, Finance].",
    limitations="Output must be a single word and case-sensitive (e.g., 'Hardware').",
    style="Raw text output. Use the following example:",
    query=f"""
    Example Input: "The market closed high."
    Example Output: Finance

    Classify this input: "{classification_text}"
    """
)

print("\n--- ENGINEERED PROMPT (gemini_model) ---")
print(create_roles_chain(gemini_model).invoke({"input": roles_classify_prompt}))

--- BAD PROMPT (Gemini 2.5 Flash) ---
The text is about **a positive outcome of a driver update for a graphics card (GPU)**. Specifically, it highlights that the update resolved latency problems, leading to a significant improvement in frame rates (performance).


--- ENGINEERED PROMPT (gemini_model) ---
Software



Prompt Set 6: Sentiment Analysis

In [39]:
sentiment_review = "The product works exactly as advertised, but the delivery took far too long, making the overall experience frustrating."


# --- R.O.L.E.S. ENGINEERED PROMPT ---
roles_sentiment_prompt = generate_roles_prompt(
    role="A Customer Service Manager focused on customer feedback.",
    objective="Analyze the provided review text to determine overall customer sentiment (Positive, Negative, or Neutral).",
    limitations="Only use the exact labels: 'Positive', 'Negative', or 'Neutral'.",
    style="Strictly output a JSON object.",
    query=f"""
    Analyze this review: "{sentiment_review}"

    Required JSON format:
    {{
      "sentiment": "<LABEL>",
      "confidence": "<1-100 score>"
    }}
    """
)

print("\n--- ENGINEERED PROMPT (gemini_model) ---")
print(create_roles_chain(gemini_model).invoke({"input": roles_sentiment_prompt}))



--- ENGINEERED PROMPT (gemini_model) ---
```json
{
  "sentiment": "Negative",
  "confidence": "85"
}
```
